In [ ]:
pip install vit-pytorch

In [ ]:
!wget https://raw.githubusercontent.com/summitgao/CAMixer/main/preclassify.py
!unzip "/content/drive/MyDrive/dataset.zip"

In [3]:
import numpy as np
from PIL import Image
import skimage
from skimage import io, measure
import random
import scipy.io as sio
import matplotlib
import matplotlib.pyplot as plt
from preclassify import del2, srad, dicomp, FCM, hcluster
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import cv2
from collections import  Counter
'''
im1_path  = '/content/Chaohu2_1.bmp'
im2_path  = '/content/Chaohu2_2.bmp'
imgt_path = '/content/Chaohu2_gt.bmp'
'''
'''
im1_path  = '/content/chaohu1/chaohu1_1.bmp'
im2_path  = '/content/chaohu1/chaohu1_2.bmp'
imgt_path = '/content/chaohu1/result.bmp'
'''
im1_path  = '/content/dataset/dataset/YellowRiver_1.bmp'
im2_path  = '/content/dataset/dataset/YellowRiver_2.bmp'
imgt_path = '/content/dataset/dataset/YellowRiver_gt.bmp'
# important parameter
patch_size = 9

In [4]:
def image_normalize(data):
  import math
  _mean = np.mean(data)
  _std = np.std(data)
  npixel = np.size(data) * 1.0
  min_stddev = 1.0 / math.sqrt(npixel)
  return (data - _mean) / max(_std, min_stddev)

def image_padding(data,r):
  if len(data.shape)==3:
    data_new=np.lib.pad(data,((r,r),(r,r),(0,0)),'constant',constant_values=0)
    return data_new
  if len(data.shape)==2:
    data_new=np.lib.pad(data,r,'constant',constant_values=0)
    return data_new
#生成自然数数组并打乱
def arr(length):
  arr=np.arange(length-1)
  #print(arr)
  random.shuffle(arr)
  #print(arr)
  return arr


# 在每个像素周围提取 patch ，然后创建成符合 pytorch 处理的格式
def createTrainingCubes(X, y, patch_size):
  # 给 X 做 padding
  margin = int((patch_size - 1) / 2)
  zeroPaddedX = image_padding(X, margin)
  # 把类别 uncertainty 的像素忽略
  ele_num1 = np.sum(y==1)
  ele_num2 = np.sum(y==2)
  patchesData_1 = np.zeros( (ele_num1, patch_size, patch_size, X.shape[2]) )
  patchesLabels_1 = np.zeros(ele_num1)

  patchesData_2 = np.zeros((ele_num2, patch_size, patch_size, X.shape[2]))
  patchesLabels_2 = np.zeros(ele_num2)

  patchIndex_1 = 0
  patchIndex_2 = 0
  for r in range(margin, zeroPaddedX.shape[0] - margin):
    for c in range(margin, zeroPaddedX.shape[1] - margin):
      # remove uncertainty pixels
      if y[r-margin, c-margin] == 1 :
        patch_1 = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
        patchesData_1[patchIndex_1, :, :, :] = patch_1
        patchesLabels_1[patchIndex_1] = y[r-margin, c-margin]
        patchIndex_1 = patchIndex_1 + 1
      elif y[r-margin, c-margin] == 2 :
        patch_2 = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
        patchesData_2[patchIndex_2, :, :, :] = patch_2
        patchesLabels_2[patchIndex_2] = y[r-margin, c-margin]
        patchIndex_2 = patchIndex_2 + 1
  patchesLabels_1 = patchesLabels_1-1
  patchesLabels_2 = patchesLabels_2-1

  #调用arr函数打乱数组
  arr_1=arr(len(patchesData_1))
  arr_2=arr(len(patchesData_2))

  train_len=10000  #设置训练集样本数
  pdata=np.zeros((train_len, patch_size, patch_size, X.shape[2]))
  plabels = np.zeros(train_len)

  for i in range(7000):
    pdata[i,:,:,:]=patchesData_1[arr_1[i],:,:,:]
    plabels[i]=patchesLabels_1[arr_1[i]]
  for j in range(7000,train_len):
    pdata[j,:,:,:]=patchesData_2[arr_2[j-7000],:,:,:]
    plabels[j]=patchesLabels_2[arr_2[j-7000]]

  return pdata, plabels
def createTestingCubes(X, patch_size):
  # 给 X 做 padding
  margin = int((patch_size - 1) / 2)
  zeroPaddedX = image_padding(X, margin)
  patchesData = np.zeros( (X.shape[0]*X.shape[1], patch_size, patch_size, X.shape[2]) )
  patchIndex = 0
  for r in range(margin, zeroPaddedX.shape[0] - margin):
    for c in range(margin, zeroPaddedX.shape[1] - margin):
      patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
      patchesData[patchIndex, :, :, :] = patch
      patchIndex = patchIndex + 1
  return patchesData


#  Inputs:  gtImg  = ground truth image
#           tstImg = change map
#  Outputs: FA  = False alarms
#           MA  = Missed alarms
#           OE  = Overall error
#           PCC = Overall accuracy
def evaluate(gtImg, tstImg):
  gtImg[np.where(gtImg>128)] = 255
  gtImg[np.where(gtImg<128)] = 0
  tstImg[np.where(tstImg>128)] = 255
  tstImg[np.where(tstImg<128)] = 0
  [ylen, xlen] = gtImg.shape
  FA = 0
  MA = 0
  label_0 = np.sum(gtImg==0)
  label_1 = np.sum(gtImg==255)
  print(label_0)
  print(label_1)

  for j in range(0,ylen):
    for i in range(0,xlen):
      if gtImg[j,i]==0 and tstImg[j,i]!=0 :
        FA = FA+1
      if gtImg[j,i]!=0 and tstImg[j,i]==0 :
        MA = MA+1

  OE = FA+MA
  PCC = 1-OE/(ylen*xlen)
  PRE=((label_1+FA-MA)*label_1+(label_0+MA-FA)*label_0)/((ylen*xlen)*(ylen*xlen))
  KC=(PCC-PRE)/(1-PRE)
  print(' Change detection results ==>')
  print(' ... ... FP:  ', FA)
  print(' ... ... FN:  ', MA)
  print(' ... ... OE:  ', OE)
  print(' ... ... PCC: ', format(PCC*100, '.2f'))
  print(' ... ... KC: ', format(KC*100, '.2f'))

def postprocess1(res):
  res_new = res
  res = measure.label(res, connectivity=2)
  #print(res)
  num = res.max()
  #print(num)
  for i in range(1, num+1):
    idy, idx = np.where(res==i)
    if len(idy) <= 20:
      res_new[idy, idx] = 0
  return res_new
def postprocess(res):
  res_new = res
  res = measure.label(res, connectivity=2)
  #print(res)
  num = res.max()
  #print(num)
  for i in range(1, num+1):
    idy, idx = np.where(res==i)
    if len(idy) <= 20:
      res_new[idy, idx] = 0.5
  return res_new

In [5]:
# read image, and then tranform to float32

im1 = io.imread(im1_path).astype(np.float32)
im2 = io.imread(im2_path).astype(np.float32)
'''
print(im1.shape)
print(im2.shape)
print(type(im1))

'''
im_gt = io.imread(imgt_path).astype(np.float32)

'''
im_gt = Image.open(imgt_path)
im_gt = np.array(im_gt)
np.unique(im_gt)

'''
im_di = dicomp(im1, im2)
ylen, xlen = im_di.shape
pix_vec = im_di.reshape([ylen*xlen, 1])

# hiearchical FCM clustering
# in the preclassification map,
# pixels with high probability to be unchanged are labeled as 1
# pixels with high probability to be changed are labeled as 2
# pixels with uncertainty are labeled as 1.5
preclassify_lab = hcluster(pix_vec, im_di)
#preclassify_lab = postprocess(preclassify_lab - 1) + 1
print('... ... hiearchical clustering finished !!!')


mdata = np.zeros([im1.shape[0], im1.shape[1], 3], dtype=np.float32)
mdata[:,:,0] = im1
mdata[:,:,1] = im2
mdata[:,:,2] = im_di
mlabel = preclassify_lab

x_train, y_train = createTrainingCubes(mdata, mlabel, patch_size)
x_train = x_train.transpose(0, 3, 1, 2)
print('... x train shape: ', x_train.shape)
print('... y train shape: ', y_train.shape)


x_test = createTestingCubes(mdata, patch_size)
x_test = x_test.transpose(0, 3, 1, 2)
print('... x test shape: ', x_test.shape)

... ... 1st round clustering ... ...
... ... 2nd round clustering ... ...
... ... hiearchical clustering finished !!!
... x train shape:  (10000, 3, 9, 9)
... y train shape:  (10000,)
... x test shape:  (74273, 3, 9, 9)


In [6]:
""" Training dataset"""
class TrainDS(torch.utils.data.Dataset):
  def __init__(self):
    self.len = x_train.shape[0]
    self.x_data = torch.FloatTensor(x_train)
    self.y_data = torch.LongTensor(y_train)
  def __getitem__(self, index):
    # 根据索引返回数据和对应的标签

    #x=torch.FloatTensor(data_rotate(self.x_data[index].cpu().numpy()))
    #y=torch.FloatTensor(gasuss_noise(self.y_data[index]))
    #x=torch.FloatTensor(datarotate(self.x_data[index]))
    #return x,self.y_data[index]
    return self.x_data[index], self.y_data[index]
  def __len__(self):
    # 返回文件数据的数目
    return self.len

# 创建 trainloader 和 testloader
trainset = TrainDS()
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=128, shuffle=True, num_workers=2)

In [7]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
# Training settings
epochs = 10
lr = 3e-5
gamma = 0.7
seed = 42
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True

seed_everything(seed)
device = 'cuda'

In [10]:
import torch
from torch import nn
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
def pair(t):
  return t if isinstance(t, tuple) else (t, t)
class Attention(nn.Module):
  def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
    super().__init__()
    inner_dim = dim_head *  heads
    project_out = not (heads == 1 and dim_head == dim)
    self.heads = heads
    self.scale = dim_head ** -0.5
    self.attend = nn.Softmax(dim = -1)
    self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

    self.to_out = nn.Sequential(
        nn.Linear(inner_dim, dim),
        nn.Dropout(dropout)
    ) if project_out else nn.Identity()
  def forward(self, x):
    qkv = self.to_qkv(x).chunk(3, dim = -1)
    q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)
    dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
    attn = self.attend(dots)
    out = torch.matmul(attn, v)
    out = rearrange(out, 'b h n d -> b n (h d)')
    return self.to_out(out)

class ViT(nn.Module):
  def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
    super().__init__()
    image_height, image_width = pair(image_size)
    patch_height, patch_width = pair(patch_size)
    assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'
    num_patches = (image_height // patch_height) * (image_width // patch_width)
    patch_dim = channels * patch_height * patch_width
    assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'
    self.to_patch_embedding = nn.Sequential(
        Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
        nn.Linear(patch_dim, dim),
    )
    self.transformer = Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)
    self.reshape = Rearrange('b (h w) (p1 p2 c) -> b c (h p1) (w p2)', p1 = patch_height, p2 = patch_width, h = image_height // patch_height)

  def forward(self, img):
    x = self.to_patch_embedding(img)
    x = self.transformer(x)
    #print(x.shape)
    x = self.reshape(x)
    #print(x.shape)
    return x
vit = ViT(
  image_size = 9,
  patch_size = 3,
  num_classes = 2,
  dim = 27,
  depth = 6,
  heads = 4,
  mlp_dim = 32,
).to(device)

In [16]:
def shift(y, n = 6):
  B, C, H, W = y.shape
  num = C // n
  out = torch.zeros_like(y)
  out[:, num * 0:num * 1, 1:, :] = y[:, num * 0:num * 1, :-1, :]  # shift down
  out[:, num * 1:num * 2, :-1, :] = y[:, num * 1:num * 2, 1:, :]  # shift up
  out[:, num * 2:num * 3, :, :-1] = y[:, num * 2:num * 3, :, 1:]  # shift left
  out[:, num * 3:num * 4, :, 1:] = y[:, num * 3:num * 4, :, :-1]  # shift right
  out[:, num * 4:, :, :] = y[:, num * 4:, :, :]  # no shift
  return out
class CSC(nn.Module):
  def __init__(self):
    super(CSC, self).__init__()
    self.conv1 = nn.Conv2d(3,12,1,1)
    self.shift = shift
    self.conv2 = nn.Conv2d(12,3,1,1)
  def forward(self, x):
    x = self.conv1(x)
    x = shift(x)
    x = self.conv2(x)
    return x
class SGU(nn.Module):
  def __init__(self, dim = 3):
    super(SGU, self).__init__()
    self.catConv =  nn.Conv2d(6, 3, kernel_size=1)
    self.norm1 = nn.LayerNorm([3, patch_size, patch_size])
    self.conv = nn.Conv2d(3,8,1,1)
    self.project_in = nn.Conv2d(3, 16, kernel_size=1)
    self.dwconv = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1, groups=16, bias=False)
    self.project_out = nn.Conv2d(8, 3, kernel_size=1)
  def forward(self, x):
    catOut = self.catConv(x)
    x = catOut
    x = self.norm1(x)
    x = self.project_in(x)
    x1, x2 = self.dwconv(x).chunk(2, dim=1)
    x = F.gelu(x1) * x2
    catOut = self.conv(catOut)
    x = x + catOut
    x = self.project_out(x)
    return x
class CAMixer(nn.Module):
  def __init__(self):
    super(CAMixer, self).__init__()
    self.vit = vit
    self.csc = CSC()
    self.sgu = SGU()
    self.linear1=nn.Linear(patch_size * patch_size * 3, 20)
    self.linear2=nn.Linear(20, 2)
  def forward(self, img):
    in_x = img.reshape(img.shape[0],-1)
    vitOut = self.vit(img)
    vitout = vitOut.reshape(vitOut.shape[0],1,-1)
    cscOut = self.csc(img)
    cscout = cscOut.reshape(cscOut.shape[0],1,-1)
    catOut = torch.cat((vitOut, cscOut), 1)
    catout = catOut.reshape(catOut.shape[0],1,-1)
    x = self.sgu(catOut)
    sguout = x.reshape(x.shape[0],1,-1)
    out1 = x.view(x.size(0), -1)     #128 1470
    out1 = self.linear1(out1)
    out1 = self.linear2(out1)
    return in_x, vitout, cscout, catout, sguout, out1

In [17]:
# loss function
#criterion = SCELoss(0.2,1)
criterion = nn.CrossEntropyLoss()
# optimizer
model = CAMixer().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [ ]:
outputs = np.zeros((ylen, xlen))
for epoch in range(25):
  epoch_loss = 0
  epoch_accuracy = 0
  for data, label in tqdm(train_loader):
    data = data.to(device)
    label = label.to(device)

    in_x, vitout, cscout, catout, sguout, output = model(data)
    loss = criterion(output, label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    acc = (output.argmax(dim=1) == label).float().mean()
    epoch_accuracy += acc / len(train_loader)
    epoch_loss += loss / len(train_loader)
  print(
      f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f}\n"
  )

In [ ]:
istrain=False
model.eval()
outputs = np.zeros((ylen, xlen))
for i in range(ylen):
  for j in range(xlen):
    if preclassify_lab[i, j] != 1.5 :
      outputs[i, j] = preclassify_lab[i, j]
    else:
      img_patch = x_test[i*xlen+j, :, :, :]
      img_patch = img_patch.reshape(1, img_patch.shape[0], img_patch.shape[1], img_patch.shape[2])
      img_patch = torch.FloatTensor(img_patch).to(device)
      #img_patch = torch.FloatTensor(img_patch)
      in_x, vitout, cscout, catout, sguout, prediction = model(img_patch)
      prediction = np.argmax(prediction.detach().cpu().numpy(), axis=1)
      outputs[i, j] = prediction+1
  if (i+1) % 50 == 0:
    print('... ... row', i+1, ' handling ... ...')

outputs = outputs-1

plt.imshow(outputs, 'gray')


res = outputs*255
res = postprocess(res)
evaluate(im_gt, res)
plt.imshow(res, 'gray')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
